In [1]:
from model import OpinionModel
from agent import OpinionAgent

In [23]:
my_model = OpinionModel(100, 0.5, 0.5)

In [24]:
my_model.schedule._agents[0].x

-0.04305067605576707

In [25]:
for i in range(100):
    my_model.step()

In [26]:
my_model.schedule._agents[0].x

0.40157810516566983

In [27]:
my_model.datacollector.get_agent_vars_dataframe().loc[(slice(0,99), 0), :]

,,Opinion,Uncertainty,Delta X,Delta U
Step,AgentID,,,,
0,0,-0.043051,0.5,0.000000e+00,0.0
1,0,-0.066105,0.5,-2.305431e-02,0.0
2,0,-0.010223,0.5,5.588209e-02,0.0
3,0,0.051975,0.5,6.219796e-02,0.0
4,0,0.066852,0.5,1.487663e-02,0.0
...,...,...,...,...,...
95,0,0.401578,0.5,4.507929e-07,0.0
96,0,0.401578,0.5,-6.288866e-08,0.0
97,0,0.401578,0.5,-2.758272e-08,0.0
